In [ ]:
import pandas as pd
import json

In [ ]:
# Replace 'data.jsonl' with the path to your JSONL file
file_path = 'data/dev.jsonl'

# Read the JSON Lines file into a list of dictionaries
data = []
with open(file_path, 'r') as file:
    for line in file:
        data.append(json.loads(line))

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

df

In [ ]:
file_path = 'data/dev-labels.lst'  # Replace this with the actual path to your file

# Read the file
with open(file_path, 'r') as file:
    labels = file.readlines()

# Display the labels
label_list = []
for label in labels:
   label_list.append(label.strip())  # .strip() removes any leading/trailing whitespaces or newlines
df['label'] = label_list

In [ ]:
df['label'] = label_list
df

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)

In [ ]:
slot_question = []
slot_answer = []
cnt = 0
filtered_rows = []
for index,row in df.iterrows():
    wrong_ans_option = 'sol'+str(2 - int(row['label']))
    wrong_ans = row[wrong_ans_option]
    slot_question = row['goal']
    rouge_score_tmp = scorer.score(row['sol1'],row['sol2'])
    f_score_tmp = rouge_score_tmp['rougeL'].fmeasure
    if (f_score_tmp)<=0.65:
        filtered_rows.append(index)
        cnt += 1
print(cnt)

In [ ]:
filtered_df = df.loc[filtered_rows]
filtered_df.reset_index(drop=True, inplace=True)
filtered_df 

In [ ]:
slot_question = []
slot_answer = []
cnt = 0
for index,row in filtered_df.iterrows():
    wrong_ans_option = 'sol'+str(2 - int(row['label']))
    wrong_ans = row[wrong_ans_option]
    slot_q = (row['goal']+'\nsol1:'+row['sol1']+"\nsol2:"+row['sol2']).replace(wrong_ans,"")
    print(slot_q,"|",wrong_ans)
    slot_question.append(slot_q)
    slot_answer.append(wrong_ans)
    

In [22]:
filtered_df['slot_question'] = slot_question
filtered_df['slot_answer'] = slot_answer
filtered_df.to_csv("data/mc.csv")